In [2]:
import torch
import pandas as pd
from torch_geometric.data import Dataset, Data
from torch_geometric.utils.convert import to_networkx
import networkx as nx
import numpy as np
from sklearn import preprocessing


In [3]:
books = pd.read_csv("Book-Crossing\BX-Books.csv", encoding='ISO-8859-1', sep=";", on_bad_lines='skip', usecols=['ISBN',	'Book-Title', 'Book-Author', 'Year-Of-Publication',	'Publisher'])
users = pd.read_csv("Book-Crossing\BX-Users.csv", encoding='ISO-8859-1', sep=";", on_bad_lines='skip')
reviews = pd.read_csv("Book-Crossing\BX-Book-Ratings.csv", encoding='ISO-8859-1', sep=";", on_bad_lines='skip')

C:\Users\Dylan\AppData\Local\Temp/ipykernel_23560/1597746388.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  books = pd.read_csv("Book-Crossing\BX-Books.csv", encoding='ISO-8859-1', sep=";", on_bad_lines='skip', usecols=['ISBN',	'Book-Title', 'Book-Author', 'Year-Of-Publication',	'Publisher'])


In [4]:
books.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company


In [5]:
le = preprocessing.LabelEncoder()
nodes = le.fit_transform(books['ISBN'], ) 

In [6]:
user_ids = np.array(users['User-ID'])
nodes = np.concatenate((nodes, user_ids))

In [7]:
string_list = reviews['User-ID'].apply(str)
df = pd.DataFrame(string_list)
df['ISBN'] = reviews['ISBN']
df

,User-ID,ISBN
0,276725,034545104X
1,276726,0155061224
2,276727,0446520802
3,276729,052165615X
4,276729,0521795028
...,...,...
1149775,276704,1563526298
1149776,276706,0679447156
1149777,276709,0515107662
1149778,276721,0590442449


In [8]:
new_nodes = np.concatenate((df['User-ID'],df['ISBN']))
nodes = le.fit_transform(new_nodes)
starting_nodes = nodes[0:1149780] #number of rows in review dataframe
ending_nodes = nodes[1149780:2299560] 

In [9]:
#new_nodes[2299559]
#in_place = np.array(], dtype=np.int64)
len(starting_nodes)
edge_index = torch.tensor([starting_nodes, ending_nodes, reviews['Book-Rating']])


C:\Users\Dylan\AppData\Local\Temp/ipykernel_23560/1555777060.py:4: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ..\torch\csrc\utils\tensor_new.cpp:201.)
  edge_index = torch.tensor([starting_nodes, ending_nodes, reviews['Book-Rating']])


In [10]:
ending_nodes[1149779]
#len(ending_nodes)

124712

In [17]:
from transformers import AutoTokenizer
from n
#combo = [books, users]
tokens_df = pd.DataFrame(books)
for i in range(0, len(users.columns)):
    tokens_df[users.columns] = users[users.columns]
#tokens_df = tokens_df.drop(271378)
tokens = [0] * 271379
tn = torch.empty((6))
#log = open("log.txt", "w")
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased", use_fast=True)
#temp = pd.DataFrame(tokens_df[tokens_df.columns[0]].apply(str))
#for i in range (1, len(tokens_df.columns)):

for j in range(0, len(tokens_df.columns)):
    tokens_df[tokens_df.columns[j]] = tokens_df[tokens_df.columns[j]].apply(str)
    for i in range(0, 271379):
        
        tokens[i] = tokenizer.tokenize(tokens_df.loc[i].iat[j])
        out = torch.tensor(tokenizer.convert_tokens_to_ids(tokens[i]))
        tokens[i] = torch.zeros((30, ), dtype=torch.long)
        tokens[i][:out.size(0)] = out
        np.savetxt("log.txt", tokens[i])
        
        #token_df.loc[i].iat[j] 
        #tokens = tokens.to_dataframe()
        # tokens[i] = torch.tensor(tokenizer.convert_tokens_to_ids(tokens[i] )) 
        #tn[:j:i] = tokenizer.convert_tokens_to_ids(tokens[i] ) 
        
    print("J:", j)        
    tn = torch.stack(tokens)
    tokens_df[tokens_df.columns[j]] = tn
    
    
#log.close()
tokens_df.head()

J: 0


RuntimeError: The expanded size of the tensor (30) must match the existing size (31) at non-singleton dimension 0.  Target sizes: [30].  Tensor sizes: [31]

In [ ]:
mn = [0]*300
if (mn[3]) == 0:
    mn = torch.empty(6)
mn


tensor([1.0286e-38, 9.0919e-39, 9.3674e-39, 9.2755e-39, 1.4013e-43, 0.0000e+00])

In [16]:
tn

tensor([[ 5890,  2683, 22203,  ...,     0,     0,     0],
        [ 2199, 28332, 12376,  ...,     0,     0,     0],
        [ 4002, 16086,  2683,  ...,     0,     0,     0],
        ...,
        [ 4002, 16086,  2692,  ...,     0,     0,     0],
        [ 5890,  2683, 17465,  ...,     0,     0,     0],
        [ 5718,  2575,  2581,  ...,     0,     0,     0]])

In [ ]:
def get_device():
    if torch.cuda.is_available():
        device = torch.device('cuda:0')
    else:
        device = torch.device('cpu') # don't have GPU 
    return device

# convert a df to tensor to be used in pytorch
def df_to_tensor(df):
    device = get_device()
    return torch.from_numpy(df.values).float().to(device)

df_tensor = df_to_tensor(tokens_df)
#series_tensor = df_to_tensor(series)

TypeError: can't convert np.ndarray of type numpy.object_. The only supported types are: float64, float32, float16, complex64, complex128, int64, int32, int16, int8, uint8, and bool.

In [ ]:
from transformers import AutoTokenizer


tokenizer2 = AutoTokenizer.from_pretrained('bert-base-uncased', use_fast=True)
combo = [books, users]
token_df2 = pd.concat(combo)

for i in range(0, len(token_df2.columns)):
    token_df2[token_df2.columns[i]] = token_df2[token_df2.columns[i]].apply(str)
    token_df2[token_df2.columns[i]] = tokenizer2(token_df2[token_df2.columns[i]].values.tolist())
    

ValueError: Length of values (3) does not match length of index (550237)

In [ ]:
labels = list(tokens_df.columns)
labels

['ISBN',
 'Book-Title',
 'Book-Author',
 'Year-Of-Publication',
 'Publisher',
 'User-ID',
 'Location',
 'Age']

In [ ]:
books.iat[100000,2]
books.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 271379 entries, 0 to 271378
Data columns (total 5 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   ISBN                 271379 non-null  object
 1   Book-Title           271379 non-null  object
 2   Book-Author          271378 non-null  object
 3   Year-Of-Publication  271379 non-null  object
 4   Publisher            271377 non-null  object
dtypes: object(5)
memory usage: 10.4+ MB


In [ ]:
u, c = np.unique(nodes, return_counts=True)
dup = u[c > 1]
dup

array([     0,      1,      3, ..., 445823, 445825, 445831])